## ♻ **Fix Before Converting**
-----------------

In [12]:
!pip install ipyfilechooser


In [17]:
from safetensors import safe_open
from safetensors.torch import save_file, load_file
import os
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

def fix_diffusers_model_conversion(load_path: str, save_path: str):
    try:
        # Load original model
        with safe_open(str(load_path), framework="pt") as f:
            tensors = load_file(f)

        # Migrate tensors
        new_tensors = {}
        for k, v in tensors.items():
            new_key = k
            # Only fix the vae
            if 'first_stage_model.' in k:
                # Migrate q, k, v keys
                new_key = new_key.replace('.to_q.weight', '.q.weight')
                new_key = new_key.replace('.to_q.bias', '.q.bias')
                new_key = new_key.replace('.to_k.weight', '.k.weight')
                new_key = new_key.replace('.to_k.bias', '.k.bias')
                new_key = new_key.replace('.to_v.weight', '.v.weight')
                new_key = new_key.replace('.to_v.bias', '.v.bias')
            new_tensors[new_key] = v

        # Save the migrated model
        with safe_open(str(save_path), framework="pt") as f:
            save_file(new_tensors, f)
            print(f"Model saved to: {save_path}")
    except Exception as e:
        print(f"Error occurred: {e}")

# Function to handle the conversion when the button is clicked
def handle_conversion(button):
    load_path = file_chooser.selected
    save_path = save_dir_chooser.selected
    if load_path and save_path:
        fix_diffusers_model_conversion(load_path, save_path)
    else:
        print("Please select both a model file and a save directory.")

# Create file selection widgets
file_chooser = FileChooser(os.getcwd())
file_chooser.title = '<b>Choose Model File</b>'
save_dir_chooser = FileChooser(os.getcwd())
save_dir_chooser.title = '<b>Choose Save Directory and File Name</b>'
save_dir_chooser.show_only_dirs = False
convert_button = widgets.Button(description="Convert")
output = widgets.Output()

# Register button click event
convert_button.on_click(handle_conversion)

# Display widgets
display(file_chooser, save_dir_chooser, convert_button, output)


FileChooser(path='/workspace/stable-diffusion-webui/models/Stable-diffusion', filename='', title='<b>Choose Mo…

FileChooser(path='/workspace/stable-diffusion-webui/models/Stable-diffusion', filename='', title='<b>Choose Sa…

Button(description='Convert', style=ButtonStyle())

Output()